# **Chapter 8: Dimensionality Reduction**

## **1. Pendahuluan**

[cite_start]Banyak masalah Machine Learning di dunia nyata melibatkan ribuan bahkan jutaan fitur untuk setiap instance pelatihan[cite: 631]. [cite_start]Banyaknya fitur ini tidak hanya membuat proses pelatihan menjadi sangat lambat, tetapi juga menyulitkan algoritma untuk menemukan solusi yang optimal[cite: 632]. [cite_start]Fenomena ini dikenal sebagai **Curse of Dimensionality** (Kutukan Dimensi)[cite: 633].

**Mengapa Bab ini Penting?**
[cite_start]Dalam banyak kasus praktis, kita dapat mengurangi jumlah fitur secara signifikan tanpa kehilangan banyak informasi[cite: 634]. [cite_start]Pengurangan dimensi membantu mempercepat pelatihan dan sangat berguna untuk visualisasi data (DataViz)[cite: 642, 643]. [cite_start]Dengan mengurangi dimensi menjadi dua atau tiga, kita dapat melihat pola seperti cluster pada grafik[cite: 644, 646].

Dalam bab ini, kita akan membahas:
* **The Curse of Dimensionality**
* **Pendekatan Utama:** Proyeksi dan Manifold Learning
* **Teknik Populer:** PCA, Kernel PCA, dan LLE

## **2. The Curse of Dimensionality**

[cite_start]Intuisi kita sering kali gagal dalam membayangkan ruang berdimensi tinggi[cite: 651]. [cite_start]Sebagai contoh, dalam hypercube 10.000 dimensi, lebih dari 99,99% titik berada sangat dekat dengan perbatasan[cite: 664, 665].

### **Sparsity dan Overfitting**
* [cite_start]Dataset berdimensi tinggi berisiko menjadi sangat jarang (*sparse*), di mana sebagian besar instance pelatihan kemungkinan besar saling berjauhan[cite: 676].
* [cite_start]Hal ini membuat prediksi menjadi kurang andal karena didasarkan pada ekstrapolasi yang jauh lebih besar dibandingkan dimensi rendah[cite: 677].
* [cite_start]Semakin banyak dimensi, semakin besar risiko model mengalami *overfitting* pada data pelatihan[cite: 678].
* [cite_start]Secara teoritis, untuk menjaga kepadatan data yang cukup, jumlah instance yang dibutuhkan tumbuh secara eksponensial terhadap jumlah dimensi[cite: 680]. [cite_start]Bahkan dengan 100 fitur, kita butuh lebih banyak instance daripada atom di alam semesta agar data tidak tersebar terlalu jarang[cite: 681].

[cite_start]Konsep dimensi ini diilustrasikan secara konseptual pada Gambar 8-1 di buku yang menunjukkan perkembangan dari titik (0D) hingga tesseract (4D)[cite: 661].

## **3. Pendekatan Utama: Proyeksi vs Manifold Learning**

### **Proyeksi (Projection)**
[cite_start]Dalam banyak masalah nyata, instance pelatihan tidak tersebar seragam, melainkan berada di dalam atau dekat dengan *subspace* berdimensi lebih rendah[cite: 685, 687]. [cite_start]Teknik proyeksi bekerja dengan memproyeksikan instance secara tegak lurus ke *subspace* tersebut[cite: 711]. [cite_start]Hal ini divisualisasikan dalam Gambar 8-2 dan 8-3 di buku, di mana data 3D diproyeksikan menjadi 2D[cite: 709, 720].

### **Manifold Learning**
[cite_start]Proyeksi tidak selalu ideal jika *subspace* terpelintir, seperti pada dataset **Swiss Roll** (Gambar 8-4)[cite: 723]. [cite_start]Memproyeksikan Swiss Roll begitu saja akan menumpuk lapisan-lapisan data[cite: 738]. [cite_start]Manifold Learning berasumsi bahwa dataset terletak dekat dengan manifold berdimensi rendah yang bisa "dibuka gulungannya" (Gambar 8-5)[cite: 739, 769].

## **4. PCA (Principal Component Analysis)**

[cite_start]PCA adalah algoritma pengurangan dimensi yang paling populer[cite: 834]. [cite_start]Tujuannya adalah mengidentifikasi *hyperplane* yang paling dekat dengan data dan memproyeksikannya ke sana[cite: 835].

### **Menjaga Varians**
[cite_start]PCA memilih sumbu yang menjaga jumlah varians maksimum, karena kemungkinan besar akan kehilangan lebih sedikit informasi (Gambar 8-7)[cite: 840, 858]. [cite_start]Sumbu ini juga meminimalkan jarak rata-rata kuadrat antara dataset asli dan proyeksinya[cite: 859].

### **Principal Components**
[cite_start]PCA menemukan sumbu yang menyumbang varians terbesar (PC pertama), lalu sumbu kedua yang ortogonal terhadap yang pertama untuk sisa varians, dan seterusnya[cite: 862, 863, 865]. [cite_start]Sumbu ke-i disebut *Principal Component* (PC) ke-i[cite: 866].

### **Implementasi Manual dengan SVD**
[cite_start]Teknik standar untuk menemukan PC adalah *Singular Value Decomposition* (SVD) yang mendekomposisi matriks X menjadi $U \Sigma V^T$[cite: 877]. [cite_start]Penting untuk diingat bahwa data harus dipusatkan (*centered*) sebelum menjalankan PCA[cite: 886].

In [ ]:
import numpy as np

# Membuat dataset 3D sederhana
np.random.seed(42)
m = 60
X = np.random.randn(m, 3)

# 1. Memusatkan data (Centering)
X_centered = X - X.mean(axis=0)

# 2. Menjalankan SVD
U, s, Vt = np.linalg.svd(X_centered)

# 3. Ekstrak dua PC pertama
c1 = Vt.T[:, 0]
c2 = Vt.T[:, 1]

print("Principal Component 1:", c1)
print("Principal Component 2:", c2)

### **Menggunakan Scikit-Learn**
[cite_start]Kelas `PCA` di Scikit-Learn secara otomatis menangani pemusatan data dan menggunakan dekomposisi SVD[cite: 903, 904].

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X2D = pca.fit_transform(X)

# Melihat rasio varians yang dijelaskan
print("Explained Variance Ratio:", pca.explained_variance_ratio_)

**Interpretasi Hasil:**
[cite_start]Atribut `explained_variance_ratio_` menunjukkan proporsi varians dataset yang terletak di sepanjang setiap komponen utama[cite: 910, 911]. [cite_start]Jika rasio PC pertama tinggi (misal 84%), berarti sebagian besar informasi asli terjaga pada sumbu tersebut[cite: 913, 915].

## **5. Memilih Jumlah Dimensi dan Kompresi**

### **Memilih Dimensi yang Tepat**
[cite_start]Alih-alih memilih angka sembarangan, lebih baik memilih jumlah dimensi yang mencakup porsi varians yang cukup besar, misalnya 95%[cite: 918]. [cite_start]Kita bisa memplot varians yang dijelaskan sebagai fungsi jumlah dimensi dan mencari "sikut" (*elbow*) pada kurva (Gambar 8-8)[cite: 928, 929].

### **PCA untuk Kompresi**
[cite_start]Setelah pengurangan dimensi, dataset memakan ruang yang jauh lebih sedikit[cite: 944]. [cite_start]Misalnya, pada MNIST, mempertahankan 95% varians mengurangi fitur dari 784 menjadi sekitar 154 fitur saja[cite: 945, 946]. [cite_start]Kita dapat melakukan rekonstruksi kembali ke dimensi asli menggunakan transformasi invers, meski akan ada sedikit kehilangan informasi yang disebut *reconstruction error*[cite: 949, 951].

In [ ]:
# Contoh menemukan jumlah komponen untuk 95% varians
pca = PCA(n_components=0.95)
X_reduced = pca.fit_transform(X)

print("Jumlah komponen terpilih:", pca.n_components_)

## **6. Variasi PCA**

* **Randomized PCA:** [cite_start]Algoritma stokastik yang lebih cepat untuk menemukan perkiraan komponen utama jika jumlah dimensi target (d) jauh lebih kecil dari jumlah fitur asli (n)[cite: 976, 977].
* **Incremental PCA (IPCA):** [cite_start]Memungkinkan pelatihan dalam mini-batch, sangat berguna jika dataset terlalu besar untuk masuk ke memori (RAM)[cite: 981, 983].
* **Kernel PCA (kPCA):** [cite_start]Menggunakan *kernel trick* untuk melakukan proyeksi non-linear yang kompleks[cite: 1004, 1005]. [cite_start]Hasil kPCA dengan berbagai kernel (Linear, RBF, Sigmoid) ditunjukkan pada Gambar 8-10 di buku[cite: 1015, 1042].

## **7. LLE (Locally Linear Embedding)**

[cite_start]LLE adalah teknik Manifold Learning non-linear yang tidak mengandalkan proyeksi[cite: 1104, 1105].

### **Cara Kerja LLE**
1. [cite_start]Mengidentifikasi k tetangga terdekat untuk setiap instance[cite: 1133].
2. [cite_start]Mencoba merekonstruksi setiap instance sebagai fungsi linear dari tetangga-tetangganya (mencari bobot optimal)[cite: 1133, 1137].
3. [cite_start]Memetakan instance ke ruang dimensi rendah sambil menjaga hubungan lokal tersebut semaksimal mungkin[cite: 1142, 1149].

[cite_start]LLE sangat baik dalam "membuka gulungan" manifold yang terpelintir (Gambar 8-12)[cite: 1107, 1131]. [cite_start]Namun, LLE tidak berskala dengan baik untuk dataset yang sangat besar karena kompleksitas komputasinya yang melibatkan $m^2$[cite: 1156].

In [ ]:
from sklearn.manifold import LocallyLinearEmbedding
from sklearn.datasets import make_swiss_roll

X_sr, t = make_swiss_roll(n_samples=1000, noise=0.2, random_state=42)

lle = LocallyLinearEmbedding(n_components=2, n_neighbors=10, random_state=42)
X_reduced_lle = lle.fit_transform(X_sr)

print("Bentuk data setelah LLE:", X_reduced_lle.shape)

## **8. Teknik Lain dan Kesimpulan**

[cite_start]Beberapa teknik populer lainnya (Gambar 8-13) meliputi[cite: 1176]:
* **MDS:** [cite_start]Menjaga jarak antar instance[cite: 1166].
* **Isomap:** [cite_start]Menjaga jarak *geodesic* (jarak terpendek melalui manifold) antar instance[cite: 1169].
* **t-SNE:** [cite_start]Menjaga instance yang mirip tetap dekat dan yang berbeda tetap jauh; sangat populer untuk visualisasi cluster[cite: 1171, 1172].
* **LDA:** [cite_start]Algoritma klasifikasi yang mempelajari sumbu paling diskriminatif antar kelas untuk proyeksi[cite: 1173, 1174].

### **Ringkasan**
[cite_start]Dimensionality Reduction membantu mempercepat pelatihan dan mempermudah visualisasi data kompleks[cite: 642, 643]. [cite_start]Meskipun akan ada sedikit kehilangan informasi dan penambahan kompleksitas pada pipeline, teknik seperti PCA dan LLE sangat krusial dalam kotak peralatan ilmuwan data[cite: 638, 639].

Bab ini telah membekali kita dengan pemahaman mendalam tentang cara menangani data berdimensi tinggi sebelum melanjutkan ke topik-topik berikutnya dalam Machine Learning.